In [1]:
import numpy as np 
import json 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from scipy import sparse 
import pickle 

### prep data

In [2]:
train_path = 'train.csv'
val_path = 'val.csv'
test_path = 'test.csv'

train = pd.read_csv(train_path)
val = pd.read_csv(val_path)
test = pd.read_csv(test_path)

In [3]:
for i in [train, val, test]:
    print(i.shape)
for i in [train, val, test]:
    i.dropna(inplace=True)

(896945, 15)
(98119, 15)
(126865, 15)


In [4]:
for i in [train, val, test]:
    print(i.shape)

(896945, 15)
(98119, 15)
(126864, 15)


In [5]:
# transform the columns to be predicted into binary values 
def transform(input):
    if input == 0:
        return 0 
    else:
        return 1

train['funny_label'] = train['funny'].apply(transform)
val['funny_label'] = val['funny'].apply(transform)
test['funny_label'] = test['funny'].apply(transform)

train['cool_label'] = train['cool'].apply(transform)
val['cool_label'] = val['cool'].apply(transform)
test['cool_label'] = test['cool'].apply(transform)

train['useful_label'] = train['useful'].apply(transform)
val['useful_label'] = val['useful'].apply(transform)
test['useful_label'] = test['useful'].apply(transform)

In [32]:
# undersample the majority class, compute the target majority value
# to keep a ratio of minority : majority = 4 : 6
target_funny = int(train['funny_label'].value_counts().to_dict()[1] * (6/4))
target_cool = int(train['cool_label'].value_counts().to_dict()[1] * (6/4))
print(target_funny, target_cool)

176583 225694


In [ ]:
# split into majority and minority
majority_funny = train[train['funny_label'] == 0]
majority_cool = train[train['cool_label'] == 0]
minority_funny = train[train['funny_label'] == 1]
minority_cool = train[train['cool_label'] == 1]

In [35]:
# random undersampling of majority class
undersample_majority_funny = majority_funny.sample(target_funny, random_state = 1)
undersample_majority_cool = majority_cool.sample(target_cool, random_state = 1)

In [43]:
# concat undersampled majority with minority to form new train df
undersample_funny = pd.concat([undersample_majority_funny, minority_funny], ignore_index=True)
undersample_cool = pd.concat([undersample_majority_cool, minority_cool], ignore_index=True)

In [45]:
# add in column in train to indicate whether the record is sampled for training for funny and cool 
sampled_funny_id = list(undersample_funny['review_id'])
sampled_cool_id = list(undersample_cool['review_id'])

def check_funny(id):
    if id in sampled_funny_id:
        return 1
    else:
        return 0 

def check_cool(id):
    if id in sampled_cool_id:
        return 1
    else:
        return 0 


In [ ]:
# create two columns in the overall training data: funny_sampled_flag and cool_sampled_flag to indicate whether the row is sampled into training dataset or not 
import pandas as pd 

sampled_funny_id = list(undersample_funny['review_id'])
sampled_cool_id = list(undersample_cool['review_id'])

def check_funny(id):
    if id in sampled_funny_id:
        return 1
    else:
        return 0 

def check_cool(id):
    if id in sampled_cool_id:
        return 1
    else:
        return 0 


train['funny_sampled_flag'] = train['review_id'].apply(check_funny)
train['cool_sampled_flag'] = train['review_id'].apply(check_cool)
train.to_csv('data/train_processed.csv')
val.to_csv('data/val_processed.csv')
test.to_csv('data/test_processed.csv')